In [5]:
import importlib
import sys
import pickle

import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["TORCH_NUM_THREADS"] = "1"

import torch

sys.path.insert(0, '..')
sys.path.insert(0, '../..')
sys.path.insert(0, '../../..')
sys.path.insert(0, '../../../..')

from model.dropout_uncertainty_enc_dec_LSTM.dropout_uncertainty_model import DropoutUncertaintyEncoderDecoderLSTM

In [6]:
#load model
file_path_model = '../../../training_variational_dropout/Repair_shop/Repair_full_grad_norm_proactive_conf_check_01_3.pkl'
model = DropoutUncertaintyEncoderDecoderLSTM.load(file_path_model, dropout=0.1)

# Load the dataset
file_path_data_set = '../../../../../../risk_controlled_proactive_conformance_checking_dev/data/encoded_data/Repair/Repair_all_normal_3_val.pkl'
repair_test_dataset = torch.load(file_path_data_set, weights_only=False)

Data set categories:  ([('concept:name', 9, {'ACKNOWLEDGEMENT': 1, 'CREATE_INVOICE': 2, 'DISASSEMBLY': 3, 'EOS': 4, 'QUALITY_CONTROL': 5, 'RECEPTION': 6, 'REPAIR': 7, 'SHIPPING': 8}), ('org:resource', 52, {'0.0': 1, '1.0': 2, '10.0': 3, '11.0': 4, '12.0': 5, '13.0': 6, '14.0': 7, '15.0': 8, '16.0': 9, '17.0': 10, '18.0': 11, '19.0': 12, '2.0': 13, '20.0': 14, '21.0': 15, '22.0': 16, '23.0': 17, '24.0': 18, '25.0': 19, '26.0': 20, '27.0': 21, '28.0': 22, '29.0': 23, '3.0': 24, '30.0': 25, '31.0': 26, '32.0': 27, '33.0': 28, '34.0': 29, '35.0': 30, '36.0': 31, '37.0': 32, '38.0': 33, '39.0': 34, '4.0': 35, '40.0': 36, '41.0': 37, '42.0': 38, '43.0': 39, '44.0': 40, '45.0': 41, '46.0': 42, '47.0': 43, '48.0': 44, '49.0': 45, '5.0': 46, '6.0': 47, '7.0': 48, '8.0': 49, '9.0': 50, nan: 51})], [('case_elapsed_time', 1, {}), ('event_elapsed_time', 1, {}), ('day_in_week', 1, {}), ('seconds_in_day', 1, {})])
Encoder input features:  [['concept:name', 'org:resource'], ['case_elapsed_time', 'even

In [7]:
import evaluation.probabilistic_evaluation
importlib.reload(evaluation.probabilistic_evaluation)
from evaluation.probabilistic_evaluation import ProbabilisticEvaluation

# device = torch.device("cuda")  # Use the first available GPU
# model = model.to(device)

new_eval = ProbabilisticEvaluation(model, 
                                   repair_test_dataset,
                                   # Change here the number of cores for workers:
                                   num_processes=128,
                                   growing_num_values = ['case_elapsed_time'],
                                   # Change here number of samples:
                                   samples_per_case = 100,
                                   sample_argmax = False,
                                   use_variance_cat = True,
                                   use_variance_num = True,
                                   all_cat= ['concept:name'],
                                   # all_num= ['case_elapsed_time', 'event_elapsed_time'])
                                   all_num= ['case_elapsed_time'])

In [8]:
def save_chunk(results, i):
    chunk_number = (i + 1)
    filename = os.path.join(output_dir, f'results_part_{chunk_number:03d}.pkl')
    with open(filename, 'wb') as f:
        pickle.dump(results, f)
    print(f"Saved {len(results)} results to {filename}")

output_dir = '../../../../../../../data/repair_shop/proact_conf_check/eval_validation_set/'

save_every = 50

results = {}
for i, (case_name, prefix_len, prefix, predicted_suffixes, suffix, mean_prediction) in enumerate(new_eval.evaluate_multi_processing(random_order=False, include_model_states=False)):
    # print(case_name, prefix_len)
    assert((case_name, prefix_len) not in results)
    results[(case_name, prefix_len)] = (prefix, suffix, mean_prediction, predicted_suffixes)
    # print(prefix_len, len(suffix))
    if (i + 1) % save_every == 0:
        save_chunk(results, i)
        results = {}

if len(results):
    save_chunk(results, i)

  0%|          | 0/3000 [00:00<?, ?it/s]

Saved 50 results to ../../../../../../../data/repair_shop/proact_conf_check/eval_validation_set/results_part_050.pkl
Saved 50 results to ../../../../../../../data/repair_shop/proact_conf_check/eval_validation_set/results_part_100.pkl
Saved 50 results to ../../../../../../../data/repair_shop/proact_conf_check/eval_validation_set/results_part_150.pkl
Saved 50 results to ../../../../../../../data/repair_shop/proact_conf_check/eval_validation_set/results_part_200.pkl
Saved 50 results to ../../../../../../../data/repair_shop/proact_conf_check/eval_validation_set/results_part_250.pkl
Saved 50 results to ../../../../../../../data/repair_shop/proact_conf_check/eval_validation_set/results_part_300.pkl
Saved 50 results to ../../../../../../../data/repair_shop/proact_conf_check/eval_validation_set/results_part_350.pkl
Saved 50 results to ../../../../../../../data/repair_shop/proact_conf_check/eval_validation_set/results_part_400.pkl
Saved 50 results to ../../../../../../../data/repair_shop/proact